# Машины Тьюринга

Реализуем основные функции

In [1]:
def E(tape: list, p: int):
    """erase"""
    tape[p] = 0
    return p

def W(tape: list, p: int):
    """write"""
    tape[p] = 1
    return p

def L(tape: list, p: int):
    """left"""
    return p - 1

def R(tape: list, p: int):
    """right"""
    return p + 1

def execute(tape:list, 
            instructions:list, 
            start_position=None, 
            start_state_index:int=1,
            ):
    if start_position is None:
        start_position = len(tape) // 2
    p = start_position
    act, state = instructions[start_state_index](tape[p])
    n_steps = 0
    while True:
        n_steps += 1
        if act is None:
            break
        p = act(tape, p)
        value = tape[p]
        act, state = instructions[state](value)
    print(f"steps count: {n_steps}")
    return p

# Запись в трех клетках

Напишем программу, которая производит запись в трех последовательных клетках и завершается, указывая на крайний левый символ. 

<img src="https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_program.png">

Стартовым состоянием будем считать $q_1$:

In [2]:
type_three_strokes = [
    None,
    lambda value : (L, 2) if value==1 else (W, 1),          # q_1
    lambda value : (L, 3) if value==1 else (W, 2),          # q_2
    lambda value : (None, None) if value==1 else (W, 3),    # q_3
]

In [5]:
tape = [0] * 10
print(tape)
start_position = 5
final_position = execute(tape, type_three_strokes, start_position)
tape, final_position

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
steps count: 6


([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], 3)

# Удваивание числа

Будем использовать **унарную систему счисления** (*unary numeral system* или *tally numeration system*), в которой, для хранения, скажем, числа 5, будет блок из пяти заштрихованных клеток: $\ldots 0111110 \ldots$. Граф перехода может быть представлен так:

![fig](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_doubling.png)

In [29]:
doubling_number_of_strokes = [
    None,
    lambda value : (L, 2)  if value else (None, None),
    lambda value : (L, 3),
    lambda value : (L, 4)  if value else (W, 3),
    lambda value : (R, 5)  if value else (W, 4),
    lambda value : (R, 5)  if value else (R, 6),
    lambda value : (R, 6)  if value else (L, 7),
    lambda value : (E, 7)  if value else (L, 8),
    lambda value : (L, 9)  if value else (L, 11),
    lambda value : (L, 9)  if value else (L, 10),
    lambda value : (L, 10) if value else (R, 2),
    lambda value : (L, 11) if value else (R, 12),
    lambda value : (None, None),
]

In [30]:
tape = [0] * 25
number = 4
position = 15
tape[position: position+number] = [1] * number
print(tape)

steps = execute(tape, doubling_number_of_strokes, position)
print(tape)
print(steps)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
steps count: 98
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6


# Произведение двух натуральных чисел

![fig](https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/images/science/turing_machine_multiplication.png)

In [31]:
multiplying = [
    None,
    lambda value : (E, 1)  if value else (R, 2),
    lambda value : (R, 3)  if value else (R, 15),
    lambda value : (R, 3)  if value else (R, 4),
    lambda value : (E, 5)  if value else (R, 4),
    lambda value : (R, 6), 
    lambda value : (R, 7)  if value else (W, 12),
    lambda value : (R, 7)  if value else (R, 8),
    lambda value : (R, 9)  if value else (W, 10),
    lambda value : (R, 9)  if value else (W, 10),
    lambda value : (L, 10) if value else (L, 11),
    lambda value : (L, 11) if value else (R, 4),
    lambda value : (L, 13),
    lambda value : (L, 14) if value else (L, 13),
    lambda value : (L, 14) if value else (R, 1),
    lambda value : (L, 17) if value else (W, 16),
    lambda value : (R, 15),
    lambda value : (L, 17) if value else (R, 18),
    lambda value : (None, None),
]

In [32]:
number1 = 4
number2 = 3

tape = [0] * 25
position = 5

# запись первого числа на ленту
tape[position: position+number1] = [1] * number1

# запись второго числа на ленту
p1 = position + number1 + 1
p2 = p1 + number2
tape[p1: p2] = [1] * number2

print(tape)

position = execute(tape, multiplying, position)
print(tape)
print(f"{position = }")
sum(tape)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
steps count: 165
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
position = 10


12